In [114]:
import os
import json
import requests
from web3 import Web3
from eth_abi.packed import encode_packed

# URL for Alchemy API
ALCHEMY_URL = f'https://polygon-mainnet.g.alchemy.com/v2/{os.getenv("ALCHEMY_API_KEY")}'

w3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))
w3.is_connected()

# Tokens info
brla_token = {
    "address": "0xE6A537a407488807F0bbeb0038B79004f19DDDFb",
    "symbol": "BRLA",
    "name": "BRLA Token",
    "currency": "BRL",
    "decimals": 18,
}
usdc_token = {
    "address": "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174",
    "symbol": "USDC",
    "name": "USD Coin (POS)",
    "currency": "USD",
    "decimals": 6,
}

# Quickswap's quoter address
QUOTER_ADDRESS = "0xa15F0D7377B2A0C0c10db057f641beD21028FC89"

In [115]:
# Load contract ABI from local file
with open("lib/univ3_quoter_abi.json") as f:
    CONTRACT_ABI = json.load(f)

quoter_contract = w3.eth.contract(address=QUOTER_ADDRESS, abi=CONTRACT_ABI)
if (
    quoter_contract.functions.WNativeToken().call()
    == "0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270"
):
    print("Quoter contract instantiated successfully")
else:
    print("Quoter contract instantiation failed")

Quoter contract instantiated successfully


In [116]:
# # Route: ('DAI', 500, 'cDAI', 3000, 'WETH')
# route_types = ["address", "uint24", "address"]
# route_tuple = (str(brla_token["address"]), int(100), str(usdc_token["address"]))
# route_str = (str(brla_token["symbol"]), int(100), str(usdc_token["symbol"]))

# print(f"Route: {route_str} ")

# # Test QuoteExactInput with 1 DAI Input
# input_token_amount = int(1_000 * (10 ** int(brla_token["decimals"])))

# route_encoded = encode_packed(route_types, route_tuple)

# amount_out = quoter_contract.functions.quoteExactInput(
#     route_encoded, int(input_token_amount)
# ).call()
# amount_out = amount_out / (10 ** int(usdc_token["decimals"]))

# print("Output Amount: {:.8f} USDC for 1000 BRLA".format(amount_out))

In [117]:
class QuoteConnector:
    def get_quote(self, token0, token1, amount):
        raise NotImplementedError

    def get_name(self):
        return self.__class__.__name__


class ParaswapConnector(QuoteConnector):
    BASE_URL = "https://apiv5.paraswap.io"

    def get_quote(self, token0, token1, amount, **kwargs):
        endpoint = "/prices"

        # Prepare the request payload
        params = {
            "srcToken": token0["address"],
            "srcDecimals": token0["decimals"],
            "destToken": token1["address"],
            "destDecimals": token1["decimals"],
            "amount": amount,
            "side": kwargs.get("side", "SELL"),  # Default to "SELL"
            "network": kwargs.get("network", "1"),  # Default to "Mainnet"
        }

        # Removing None values
        params = {k: v for k, v in params.items() if v is not None}

        response = requests.get(self.BASE_URL + endpoint, params=params)
        data = response.json()

        # Verify the successful response structure and extract the destination amount
        if "priceRoute" in data and "destAmount" in data["priceRoute"]:
            return data["priceRoute"]["destAmount"]
        else:
            raise ValueError(f"Unexpected response format from Paraswap: {data}")


# Defining connector
connector = ParaswapConnector()


# # Testing liquidity depth for various amounts
# def check_liquidity_various_depth(connector, token0, token1, amounts, base_rate):
#     for amount in amounts:
#         absolute_amount = int(amount * (10 ** int(token0["decimals"])))
#         try:
#             quote = connector.get_quote(
#                 token0, token1, amount=absolute_amount, network="137"
#             )
#             adjusted_quote = int(quote) / (10 ** int(token1["decimals"]))
#             quote_rate = adjusted_quote / amount
#             comparison = ((quote_rate - base_rate) / base_rate) * 100
#             print(
#                 f"{amount} {token0['symbol']} = {adjusted_quote:.3f} {token1['symbol']}, Quote Rate: {quote_rate:.3f}, Versus Base Rate: {comparison:.3f}%"
#             )
#         except ValueError as e:
#             print(
#                 f"Not enough liquidity for {amount} {token0['symbol']}. Error: {str(e)}"
#             )


# # Getting the base currency rate for BRL-USD
# response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
# data = response.json()

# brl_rate = data["rates"]["BRL"]
# print(f"BASE RATE: 1 USD = {brl_rate:.3f} BRL")

# check_liquidity_various_depth(
#     connector, usdc_token, brla_token, [1, 100, 1000, 2000, 10000], base_rate=brl_rate
# )

# # Getting the base currency rate for USD-BRL
# response = requests.get("https://api.exchangerate-api.com/v4/latest/BRL")
# data = response.json()

# usd_rate = data["rates"]["USD"]
# print(f"BASE RATE: 1 BRL = {usd_rate:.3f} USD")

# check_liquidity_various_depth(
#     connector, brla_token, usdc_token, [5, 500, 5000, 10000, 50000], base_rate=usd_rate
# )

In [118]:
class LiquidityCalculator:
    def __init__(self, connector, token0, token1):
        self.connector = connector
        self.token0 = token0
        self.token1 = token1
        self.amounts = [100, 1000, 2000, 5000]  # Standardized amounts
        self.base_rate = self.get_base_rate()
        print(f"Starting Liq Calc for {self.token0['symbol']}-{self.token1['symbol']}")
        print(f"Base Rate: {self.base_rate:.3f}")

    def get_base_rate(self):
        base_currency = self.token0["currency"]
        response = requests.get(
            f"https://api.exchangerate-api.com/v4/latest/{base_currency}"
        )
        data = response.json()
        return data["rates"][self.token1["currency"]]

    def check_liquidity(self):
        for amount in self.amounts:
            adjusted_amount = (
                amount / self.base_rate if self.token0["currency"] != "USD" else amount
            )
            absolute_amount = int(
                adjusted_amount * (10 ** int(self.token0["decimals"]))
            )
            try:
                quote = self.connector.get_quote(
                    self.token0, self.token1, amount=absolute_amount, network="137"
                )
                adjusted_quote = int(quote) / (10 ** int(self.token1["decimals"]))
                quote_rate = adjusted_quote / adjusted_amount
                comparison = ((quote_rate - self.base_rate) / self.base_rate) * 100
                if abs(comparison) > 1:
                    print(
                        f"🔴 -> {adjusted_amount} {self.token0['symbol']} = {adjusted_quote:.3f} {self.token1['symbol']}, Quote Rate: {quote_rate:.3f}, Vs Base Rate: {comparison:.3f}%"
                    )
                else:
                    print(
                        f"🟢 -> {adjusted_amount} {self.token0['symbol']} = {adjusted_quote:.3f} {self.token1['symbol']}, Quote Rate: {quote_rate:.3f}, Vs Base Rate: {comparison:.3f}%"
                    )
            except ValueError as e:
                print(
                    f"Not enough liquidity for {adjusted_amount} {self.token0['symbol']}. Error: {str(e)}"
                )
        print("Done!")

# Instantiate the ArbitrageCalculator for USD to BRL
usd_brl_calculator = LiquidityCalculator(connector, usdc_token, brla_token)
usd_brl_calculator.check_liquidity()

# Instantiate the ArbitrageCalculator for BRL to USD
brl_usd_calculator = LiquidityCalculator(connector, brla_token, usdc_token)
brl_usd_calculator.check_liquidity()

Starting Liq Calc for USDC-BRLA
Base Rate: 4.980
🟢 -> 100 USDC = 496.706 BRLA, Quote Rate: 4.967, Vs Base Rate: -0.260%
🟢 -> 1000 USDC = 4949.477 BRLA, Quote Rate: 4.949, Vs Base Rate: -0.613%
🟢 -> 2000 USDC = 9863.674 BRLA, Quote Rate: 4.932, Vs Base Rate: -0.967%
🔴 -> 5000 USDC = 24407.132 BRLA, Quote Rate: 4.881, Vs Base Rate: -1.979%
Done!
Starting Liq Calc for BRLA-USDC
Base Rate: 0.201
🟢 -> 497.51243781094524 BRLA = 100.054 USDC, Quote Rate: 0.201, Vs Base Rate: 0.054%
🟢 -> 4975.124378109453 BRLA = 996.820 USDC, Quote Rate: 0.200, Vs Base Rate: -0.318%
🟢 -> 9950.248756218905 BRLA = 1985.657 USDC, Quote Rate: 0.200, Vs Base Rate: -0.717%
🔴 -> 24875.621890547263 BRLA = 4878.350 USDC, Quote Rate: 0.196, Vs Base Rate: -2.433%
Done!
